In [277]:
import json
from pprint import pprint

In [278]:
with open("Data/JSON/PCS_Index.json") as index_file:
    index = json.load(index_file)
with open("Data/JSON/PCS_Tables.json") as table_file:
    tables = json.load(table_file)
with open("Data/JSON/PCS_Definitions.json") as definitions_file:
    defs = json.load(definitions_file)

In [291]:
# Testing block
# Run code blocks below before this block
letters = index["ICD10PCS.index"]["letter"]
for letter in letters:
    for mainterm in letter["mainTerm"]:
        # Search for user's lookup term
        user_mainterm = "Suture" # Only for testing purposes, user will pick
        if mainterm["title"] == user_mainterm:
            # Check to see if levels used for this term
            levels = check_for_levels(mainterm)
            if levels == "level_1":
                print("EXECUTE SINGLE LEVEL")
                execute_single_level_structure(mainterm)
            elif not levels:
                print("PARENT EXECUTE")
                parent_execute(mainterm)
            else:
                print("PROGRESS THROUGH LEVELS")
                progress_through_levels(mainterm)

EXECUTE SINGLE LEVEL
['Laceration repair', 'Ligation']
Choose term that most correlates to this medical case: Ligation
PARENT EXECUTE
Redirected to term 'Occlusion'.
FOUND ON FIRST TRY.
{'term': [{'_level': '1', 'codes': '0FLC', 'title': 'Ampulla of Vater'},
          {'_level': '1', 'codes': '0DLQ', 'title': 'Anus'},
          {'_level': '1',
           'term': [{'_level': '2', 'codes': '04L0', 'title': 'Abdominal'},
                    {'_level': '2',
                     'code': '02LW3DJ',
                     'title': 'Thoracic, Descending'}],
           'title': 'Aorta'},
          {'_level': '1',
           'term': [{'_level': '2',
                     'term': [{'_level': '3', 'codes': '04LQ', 'title': 'Left'},
                              {'_level': '3',
                               'codes': '04LP',
                               'title': 'Right'}],
                     'title': 'Anterior Tibial'},
                    {'_level': '2',
                     'term': [{'_level': '

In [286]:
def execute_single_level_structure(mainterm):
    if {"see", "_level"} == mainterm["term"][0].keys():        
        check_for_same_structure(mainterm, {"see", "_level"}) # Testing purposes only
        if isinstance(mainterm["term"][0]["see"], str): # If see tag is a string
            execute_single_level_structure_1(mainterm)   
        elif {"tab", "__text"} == mainterm["term"][0]["see"].keys():
            execute_single_level_structure_2(mainterm)
        elif {"codes", "__text"} == mainterm["term"][0]["see"].keys():
            execute_single_level_structure_2(mainterm)
    if {"title", "see", "_level"} == mainterm["term"][0].keys():
        check_for_same_structure(mainterm, {"title", "see", "_level"}) # Testing purposes only
        execute_single_level_structure_3(mainterm)    
    if {"use", "_level"} == mainterm["term"][0].keys():
        check_for_same_structure(mainterm, {"use", "_level"}) # Testing purposes only
        execute_single_level_structure_4(mainterm)
                        
def check_for_same_structure(mainterm, set_like_object):
    for i in mainterm["term"]:
        if not set_like_object == i.keys():
            print("SEE EXECUTE_LEVEL_1_STRUCTURE TO FIX -> CHECK_FOR_SAME_STRUCTURE.")
            break

def execute_single_level_structure_1(mainterm):
    text_choices = [i["see"] for i in mainterm["term"]]
    print(text_choices)
    user_input = input("Choose term that most correlates to this medical case: ")
    # Requery data to find user's choice. Assumed 'see' tag
    mainterms = (mainterms
            for letter in letters
            for mainterms in letter["mainTerm"])
    for new_mainterm in mainterms:
        if new_mainterm["title"] == user_input:
            # Check to see if levels used for this term
            levels = check_for_levels(new_mainterm)
            if levels == "level_1":
                print("SEE EXECUTE_LEVEL_1_STRUCTURE() TO FIX. ERROR 2.")
            if not levels:
                print("PARENT EXECUTE")
                parent_execute(new_mainterm)
            else:
                print("PROGRESS THROUGH LEVELS")
                progress_through_levels(new_mainterm)
            break
            
def execute_single_level_structure_2(mainterm):
    text_choices = [i["see"]["__text"] for i in mainterm["term"]]
    print(text_choices)
    user_input = input("Choose term that most correlates to this medical case: ")
    for new_mainterm in mainterm["term"]:
        if new_mainterm["see"]["__text"] == user_input:
            parent_execute(new_mainterm)
            
def execute_single_level_structure_3(mainterm):
    text_choices = [i["title"] for i in mainterm["term"]]
    print(text_choices)
    user_input = input("Choose term that most correlates to this medical case: ")
     # Requery data to find user's choice.
    for new_mainterm in mainterm["term"]:
        if new_mainterm["title"] == user_input:
            # Check to see if levels used for this term
            levels = check_for_levels(new_mainterm)
            if levels == "level_1":
                print("SEE EXECUTE_LEVEL_1_STRUCTURE() TO FIX. ERROR 2.")
            if not levels:
                print("PARENT EXECUTE")
                parent_execute(new_mainterm)
            else:
                print("PROGRESS THROUGH LEVELS")
                progress_through_levels(new_mainterm)
            break

def execute_single_level_structure_4(mainterm):
    if isinstance(mainterm["term"][0]["use"], str):
        text_choices = [i["use"] for i in mainterm["term"]]
        print(text_choices)
        user_input = input("Choose term that most correlates to this medical case: ")
        for new_mainterm in mainterm["term"]:
            if new_mainterm["use"] == user_input:
                parent_execute(new_mainterm)
    else:
        if {"tab", "__text"} == mainterm["term"][0]["use"].keys():
            text_choices = [i["use"]["__text"] for i in mainterm["term"]]
            print(text_choices)
            user_input = input("Choose term that most correlates to this medical case: ")
            for new_mainterm in mainterm["term"]:
                if new_mainterm["use"]["__text"] == user_input:
                    parent_execute(new_mainterm)
                    break

In [272]:
def progress_through_levels(new_mainterm):
    level_flag = True
    while level_flag:
        new_level_terms = get_next_level_choices(new_mainterm)
        if new_level_terms == None:
            execute_group_3(new_mainterm)
            break
        elif len(new_level_terms) == 1:
            print(f"Automatically choosing term, '{new_level_terms[0]}', because it's the only choice available.")
            new_mainterm = [i for i in new_mainterm["term"] if i["title"] == new_level_terms[0]][0]
        else:
            print(new_level_terms)
            user_input = input("Choose term : ")
            new_mainterm = [i for i in new_mainterm["term"] if i["title"] == user_input][0]
        levels = check_for_levels(new_mainterm)
        if not levels:
            # return code, tab, codes, see, use
            execute_group_3(new_mainterm)
            level_flag = False

def check_for_levels(mainterm):
    try:
        for sub_term in mainterm["term"]:
            try:
                if sub_term["term"]:
                    return True
            except KeyError as err:
                pass
        try:
            if mainterm["term"][0]["_level"]:
                return "level_1"
        except KeyError as err:
            return False
    except KeyError as err:
        return False
            
def check_for_code(mainterm):
    try:
        print(f"Use final code: {mainterm['code']}")
    except (KeyError) as err:
        return False
    
def get_next_level_choices(mainterm):
    try:
        next_level_terms = [level["title"] for level in mainterm["term"]]
    except KeyError as err:
        for term in mainterm["term"]:
            parent_execute(mainterm)
        next_level_terms = None
    return next_level_terms

In [273]:
def parent_execute(mainterm):
    if {"title", "use"} == mainterm.keys():
        execute_group_1(mainterm)
    elif {"title", "see"} == mainterm.keys():
        execute_group_2(mainterm)
    elif {"title", "term"} == mainterm.keys():
        execute_group_3(mainterm)
    elif {"title", "tab"} == mainterm.keys():
        execute_group_5(mainterm)
    elif {"title", "codes"} == mainterm.keys():
        execute_group_6(mainterm)
    elif {"title", "code"} == mainterm.keys():
        execute_group_4(mainterm)
    elif {"title", "code", "term"} == mainterm.keys():
        execute_group_7(mainterm)
    elif {"title", "code", "_level"} == mainterm.keys():
        execute_group_4(mainterm)
    elif {"see", "_level"} == mainterm.keys():
        execute_group_2(mainterm)
    elif {"use", "_level"} == mainterm.keys():
        execute_group_1(mainterm)
    elif {"title", "see", "_level"} == mainterm.keys():
        execute_group_2(mainterm)
    else:
        pprint(mainterm)
        print("###############################NO MATCHES FOUND - PARENT EXEUTE ###########################################")
          
# Functions to handle querys 
def execute_group_1(mainterm):
    if isinstance(mainterm["use"], str): # If "use" value has no children
        execute_group_1_1(mainterm)
    elif {"tab", "__text"} == mainterm["use"].keys():
        execute_group_1_2(mainterm)
        
def execute_group_2(mainterm):
    if isinstance(mainterm["see"], str): # If "see" value has no children
        execute_group_2_1(mainterm)
    elif {"codes", "__text"} == mainterm["see"].keys():
        execute_group_2_2(mainterm)
    elif {"tab", "__text"} == mainterm["see"].keys():
        execute_group_2_3(mainterm)

def execute_group_3(mainterm):
    # If user is met with a "term" tag that means there's levels of code
    if {'title', 'codes', '_level'} == mainterm.keys():
        execute_group_6(mainterm)  
    elif {'title', 'code', '_level'} == mainterm.keys():
        execute_group_4_1(mainterm)
    elif {'see', '_level'} == mainterm.keys():
        execute_group_2(mainterm)
    elif {'use', '_level'} == mainterm.keys():
        execute_group_1(mainterm)
    elif {'title', 'see', '_level'} == mainterm.keys():
        execute_group_2(mainterm)
    elif {'code', '_level'} == mainterm.keys():
        execute_group_4(mainterm)
    elif {'title', 'tab', '_level'} == mainterm.keys():
        execute_group_5(mainterm)
    else:
        pprint(mainterm)
        print("###############################NO MATCHES FOUND - EXECUTE GROUP 3 ###########################################")
def execute_group_4(mainterm):
    execute_group_4_1(mainterm)
    
def execute_group_5(mainterm):
    execute_group_5_1(mainterm)

def execute_group_6(mainterm):
    execute_group_6_1(mainterm)

def execute_group_7(mainterm):
    execute_group_7_1(mainterm)
        

In [274]:
# Detailed functions to handle queries
def execute_group_1_1(mainterm):
    print(f"Use the code associated with term '{mainterm['use']}' in the PCS Table.")

def execute_group_1_2(mainterm):
    table = mainterm["use"]["tab"]
    text = mainterm["use"]["__text"]
    print(f"Go to table {table}, located at sections {text}")

def execute_group_2_1(mainterm): #############################################################
    new_term = mainterm["see"]
    print(f"Redirected to term '{new_term}'.")
    # Requery to find new term
    mainterms = (mainterms for letter in letters for mainterms in letter["mainTerm"])
    for new_mainterm_1 in mainterms:
        if new_mainterm_1["title"] == new_term:
            print("FOUND ON FIRST TRY.")
            parent_execute(new_mainterm_1) # Found new mainterm
        else:
            split_term = new_term.split(", ")
            if len(split_term) == 2:
                new_search_parent, new_search_child = split_term
                print(f"LOOKING FOR NEW PARENT TERM: {new_search_parent}")
                for new_mainterm_2 in mainterms:
                    if new_mainterm_2["title"] == new_search_parent:
                        for subterm in new_mainterm_2["term"]:
                            print(f"LOOKING FOR NEW CHILD TERM: {new_search_child}")
                            if subterm["title"] == new_search_child:
                                levels = check_for_levels(subterm)
#                                 if levels == "level_1":
#                                     print("SEE EXECUTE_GROUP_2_1 TO FIX ERROR")
                                if not levels:
                                    parent_execute(subterm)
                                else:
                                    progress_through_levels(subterm)
            elif len(split_term) == 3:
                new_search_parent, new_search_child_1, new_search_child_2 = split_term
                print(f"LOOKING FOR NEW PARENT TERM: {new_search_parent}")
                for new_mainterm_2 in mainterms:
                    if new_mainterm_2["title"] == new_search_parent:
                        for subterm in new_mainterm_2["term"]:
                            print(f"LOOKING FOR NEW CHILD TERM: {new_search_child_1}")
                            if subterm["title"] == new_search_child_1:
                                for subterm_1 in subterm:
                                    print(f"LOOKING FOR NEW CHILD TERM: {new_search_child_2}")
                                    if subterm_1["title"] == new_search_child_2:
                                        levels = levels = check_for_levels(subterm_1)
                                        if not levels:
                                            parent_execute(subterm_1)
                                        else:
                                            progress_through_levels(subterm_1)
                        
                        
                        
def execute_group_2_2(mainterm):    
    code = mainterm["see"]["codes"]
    text = mainterm["see"]["__text"]
    print(f"""Go to table '{code[:3]}', located at '{text}'.
Position 4 and greater code(s): '{code[3:]}'
Each code corresponds to it's respective position number.""")

def execute_group_2_3(mainterm):
    table = mainterm["see"]["tab"]
    text = mainterm["see"]["__text"]
    print(f"Go to table '{table}', located at sections '{text}'.")

def execute_group_4_1(mainterm):
    print(f"Use final code: {mainterm['code']} with description '{mainterm['title']}'.")
    
def execute_group_5_1(mainterm):
    table = mainterm["tab"]
    print(f"Go to table: {table}. No additional guidance given.")
    
def execute_group_6_1(mainterm):
    code = mainterm["codes"]
    print(f"""Go to table: {code[:3]}
Position 4 and greater code(s): '{code[3:]}'
Each code corresponds to it's respective position number.""")
    
def execute_group_7_1(mainterm): ##################################################################
    code = mainterm["code"]
    level_1_terms = [term["title"]
                    for term in mainterm["term"]
                    if term["_level"] == "1"]
    user_input = input(f"""Use code: {code}. If this case involves any of the following terms, {level_1_terms}, press 'y' now. If not, press 'n' to exit.""")
    if user_input == "y":
        pass
        # Execute a lookup with levels involved
        
def execute_group_3_13(mainterm):
    if ["_level"] == list(mainterm.keys()):
        print("No term given. Search your term again and choose a different option.")

In [275]:
# All possible key combinations at letter -> mainterm -> mainterm.keys()
group_1 = ["title", "use"]
group_2 = ['title', 'see']
group_3 = ['title', 'term']
group_4 = ['title', 'code']
group_5 = ['title', 'tab']
group_6 = ['title', 'codes']
group_7 = ['title', 'code', 'term']

In [276]:
# All possible key combinations at letter -> mainterm -> term -> for i in term -> term.keys()
group_3_1 = ['see', '_level'] #
group_3_2 = ['use', '_level'] #
group_3_3 = ['title', 'codes', '_level'] #
group_3_4 = ['title', 'code', '_level'] #
group_3_5 = ['title', 'term', '_level']
group_3_6 = ['title', 'see', '_level'] #
group_3_7 = ['code', '_level'] #
group_3_9 = ['title', 'codes', 'term', '_level']
group_3_10 = ['title', 'tab', '_level'] #
group_3_11 = ['title', 'tab', 'term', '_level']
group_3_12 = ['title', 'code', 'term', '_level']
group_3_13 = ['_level'] # Outlier piece of data that doesnt contain any data. Delete? ("Radiation Therapy") #